In [4]:
import requests
import lxml.html as lh

gdelt_base_url = 'http://data.gdeltproject.org/events/'

# get the list of all the links on the gdelt file page
page = requests.get(gdelt_base_url+'index.html')
doc = lh.fromstring(page.content)
link_list = doc.xpath("//*/ul/li/a/@href")

# separate out those links that begin with four digits 
file_list = [x for x in link_list if str.isdigit(x[0:4])]

print file_list[:5]

['20170405.export.CSV.zip', '20170404.export.CSV.zip', '20170403.export.CSV.zip', '20170402.export.CSV.zip', '20170401.export.CSV.zip']


In [186]:
# read excel with country names and codes
countries = pd.ExcelFile('/Users/sabine.a.joseph/Desktop/Country_codes_NAMO.xlsx')
countries = countries.parse('Sheet1')
countries.shape

(24, 7)

In [185]:
infilecounter = 0
outfilecounter = 0

CCs = list(countries.Country_2)

In [215]:
import os.path
import urllib
import zipfile
import glob
import operator

local_path = '/Users/sabine.a.joseph/Desktop/GDELT_Data/'

fips_country_code = 'SY'

#for compressed_file in file_list[infilecounter:]:
for compressed_file in file_list[:1]:
    print compressed_file,
    
    # if we dont have the compressed file stored locally, go get it. Keep trying if necessary.
    while not os.path.isfile(local_path+compressed_file): 
        print 'downloading,',
        urllib.urlretrieve(url=gdelt_base_url+compressed_file, 
                           filename=local_path+compressed_file)
        
    # extract the contents of the compressed file to a temporary directory    
    print 'extracting,',
    z = zipfile.ZipFile(file=local_path+compressed_file, mode='r')    
    z.extractall(path=local_path+'tmp/')
    
    # parse each of the csv files in the working directory, 
    print 'parsing,',
    for infile_name in glob.glob(local_path+'tmp/*'):
        outfile_name = local_path+'country/'+fips_country_code+'%04i.tsv'%outfilecounter
        
        # open the infile and outfile
        with open(infile_name, mode='r') as infile, open(outfile_name, mode='w') as outfile:
            for line in infile:
                # extract lines with our interest country code
                #based on columns: ActionGeo_ADM1Code 51, Actor1Geo_ADM1Code 37, Actor2Geo_ADM1Code 44
                if 'IS' in operator.itemgetter(51, 37, 44)(line.split('\t')): #or\
                # CCs[1] in operator.itemgetter(51, 37, 44)(line.split('\t')):
                #if fips_country_code in operator.itemgetter(51, 37, 44)(line.split('\t')):    
                    outfile.write(line)
            outfilecounter +=1
            
        # delete the temporary file
        os.remove(infile_name)
    infilecounter +=1
    print 'done'

20170405.export.CSV.zip extracting, parsing, done


In [216]:
import glob
import pandas as pd

# Get the GDELT field names from a helper file
colnames = pd.read_excel('/Users/sabine.a.joseph/Desktop/CSV.header.fieldids.xlsx', sheetname='Sheet1', 
                         index_col='Column ID', parse_cols=1)['Field Name']

# Build DataFrames from each of the intermediary files
files = glob.glob(local_path+'country/'+fips_country_code+'*')
DFlist = []
for active_file in files:
    print active_file
    DFlist.append(pd.read_csv(active_file, sep='\t', header=None, dtype=str,
                              names=colnames))

# Merge the file-based dataframes and save a pickle
DF = pd.concat(DFlist)
DF.to_pickle(local_path+'backup'+fips_country_code+'.pickle')    

# once everythin is safely stored away, remove the temporary files
for active_file in files:
    os.remove(active_file)

/Users/sabine.a.joseph/Desktop/GDELT_Data/country/SY0005.tsv


In [217]:
print DF.shape
DF.head()

(4849, 58)


,GLOBALEVENTID,SQLDATE,MonthYear,Year,FractionDate,Actor1Code,Actor1Name,Actor1CountryCode,Actor1KnownGroupCode,Actor1EthnicCode,...,Actor2Geo_FeatureID,ActionGeo_Type,ActionGeo_FullName,ActionGeo_CountryCode,ActionGeo_ADM1Code,ActionGeo_Lat,ActionGeo_Long,ActionGeo_FeatureID,DATEADDED,SOURCEURL
0,643557563,20170306,201703,2017,2017.1808,NaN,NaN,NaN,NaN,NaN,...,-2542538,1,Israel,IS,IS,31.5,34.75,IS,20170405,http://www.fox21online.com/2017/04/04/latest-s...
1,643557591,20170306,201703,2017,2017.1808,ISR,ISRAEL,ISR,NaN,NaN,...,-2542538,4,"Idlib, Idlib, Syria",SY,SY12,35.9306,36.6339,-2542538,20170405,http://www.fox21online.com/2017/04/04/latest-s...
2,643557598,20170306,201703,2017,2017.1808,SYR,SYRIAN,SYR,NaN,NaN,...,IS,4,"Idlib, Idlib, Syria",SY,SY12,35.9306,36.6339,-2542538,20170405,http://www.fox21online.com/2017/04/04/latest-s...
3,643557627,20170329,201703,2017,2017.2438,EURGOV,EUROPEAN,EUR,NaN,NaN,...,-781545,4,"Tel Aviv, Tel Aviv, Israel",IS,IS05,32.0667,34.7667,-781545,20170405,http://english.wafa.ps/page.aspx?id=MY8mKZa724...
4,643557628,20170329,201703,2017,2017.2438,EURGOV,EUROPEAN,EUR,NaN,NaN,...,-781545,4,"Tel Aviv, Tel Aviv, Israel",IS,IS05,32.0667,34.7667,-781545,20170405,http://english.wafa.ps/page.aspx?id=MY8mKZa724...


In [87]:
df_test=DF[:100]

In [ ]:
#GDELT numerical codes for actor etc. codes
################################

#normalize each daily file and then concatenate them?
#####


### aggregation per day per country 
# x days X x countries 

# delete columns
del df_test.MonthYear
del df_test.FractionDate

del df_test.Actor1KnownGroupCode # seems unpopulated
#del DF['DATEADDED']
# DF['Actor1Geo_ADM1Code'] # country code and mess, discard
# DF['Actor1Geo_FullName'] # long names - discard 
# DF['Actor1Geo_FeatureID'] #mess - discard
# DF['Actor2Geo_FullName'] #mess - discard
# DF['Actor2Geo_ADM1Code'] #mess - discard
# DF['Actor2Geo_FeatureID'] #mess - discard
# DF['ActionGeo_FullName'] #mess - discard
# DF['ActionGeo_ADM1Code'] #mess - discard
# DF['ActionGeo_FeatureID'] #mess - discard
#####

# upper, count Actor1Code frequency per day, pick the most frequent actor for a given date, 
# vectorize

# upper, count Actor1Name frequency per day, pick the most frequent actor for a given date, 
# vectorize

# upper, count Actor1CountryCode frequency per day, pick the most frequent actor for a given date, 
# replace str code with numeric country code

# df_test['Actor1EthnicCode'] 
# upper, count Actor1Name frequency per day, pick the most frequent actor for a given date, 
# vectorize

# df_test['Actor1Religion1Code'] 
# upper, count Actor1Name frequency per day, pick the most frequent actor for a given date, 
# vectorize

#DF['Actor1Type1Code']
# upper, count Actor1Name frequency per day, pick the most frequent actor for a given date, 
# vectorize

# DF['Actor1Type2Code']
# upper, count Actor1Name frequency per day, pick the most frequent actor for a given date, 
# vectorize

# DF['Actor1Type3Code'] # observe sparsity
# DF['Actor2Code'] # messy text 
# DF['Actor2Name'] # text 

# DF['Actor2CountryCode']
# upper, count Actor1CountryCode frequency per day, pick the most frequent actor for a given date, 
# replace str code with numeric country code

# DF['Actor2KnownGroupCode']
# upper, count Actor1Name frequency per day, pick the most frequent actor for a given date, 
# vectorize

# DF['Actor2EthnicCode']
# upper, count Actor1Name frequency per day, pick the most frequent actor for a given date, 
# vectorize

# DF['Actor2Religion1Code']
# upper, count Actor1Name frequency per day, pick the most frequent actor for a given date, 
# vectorize

# DF['Actor2Religion2Code']
# upper, count Actor1Name frequency per day, pick the most frequent actor for a given date, 
# vectorize

# DF['Actor2Type2Code']
# DF['Actor2Type3Code']
# as above ones

# DF['IsRootEvent']
# binary, numeric var, count and take most frequent number 

# DF['EventCode'] - rebellion, insurgency? - what it codes for?
# numeric, take most frequent code

# DF['EventBaseCode']
# numeric, take most frequent code

# DF['EventBaseCode']
# numeric, take most frequent code

# DF['QuadClass']
# 4 possible numeric classes

# DF['GoldsteinScale'] ### very important feature
# numeric, take mean or median

# DF['NumMentions'] ### metions of what?
# numeric, take most frequent number or take mean or median

# DF['NumSources'] ### to normalize mentions?
# DF['NumArticles']
# DF['AvgTone'] ## take mean or median
# DF['Actor1Geo_Type'] # 5 possible numeric values

# DF['Actor1Geo_CountryCode'] # covert to numeric country code, then take most frequent one

# DF['Actor1Geo_Lat'] # ? linked to most frequent something else (other var)
# DF['Actor1Geo_Long'] # ?

# DF['Actor2Geo_Type'] # 5 possible types
# DF['Actor2Geo_CountryCode'] # covert to numeric country codes

# DF['Actor2Geo_Lat'] # ? linked to most frequent something else (other var)
# DF['Actor2Geo_Long']

# DF['ActionGeo_Type'] # 5 possible numeric values, take most frequent one
# DF['ActionGeo_CountryCode'] # # covert to numeric country codes

# DF['ActionGeo_Lat'] # ? linked to most frequent something else (other var)
# DF['ActionGeo_Long']

# DF['SOURCEURL'] # could take most frequent source, parse url.. but rather ignore

# new col - DF['SQLDATE'].value_counts() - sum them per day / month 

In [81]:
# df_test['counts']= df_test.groupby(['SQLDATE'])['Actor2Religion2Code'].transform('count')

In [218]:
DF[[51, 37, 44]][:20]

,ActionGeo_CountryCode,Actor1Geo_CountryCode,Actor2Geo_CountryCode
0,IS,NaN,SY
1,SY,IS,SY
2,SY,SY,IS
3,IS,IS,IS
4,IS,IS,IS
5,IS,IS,IS
6,IS,IS,IS
7,IS,IS,IS
8,IS,IS,IS
9,IS,IS,IS


In [219]:
DF['SQLDATE'].value_counts()

20170405    4675
20170306      62
20170329      56
20160405      38
20170404      16
20070408       2
Name: SQLDATE, dtype: int64

In [102]:
# test = df_test.groupby(['SQLDATE', 'Actor2Religion2Code']).agg(lambda x:x.value_counts().index[0])
df_test.groupby('SQLDATE').agg('count')

,GLOBALEVENTID,MonthYear,Year,FractionDate,Actor1Code,Actor1Name,Actor1CountryCode,Actor1KnownGroupCode,Actor1EthnicCode,Actor1Religion1Code,...,Actor2Geo_FeatureID,ActionGeo_Type,ActionGeo_FullName,ActionGeo_CountryCode,ActionGeo_ADM1Code,ActionGeo_Lat,ActionGeo_Long,ActionGeo_FeatureID,DATEADDED,SOURCEURL
SQLDATE,,,,,,,,,,,,,,,,,,,,,
20160405,1,1,1,1,1,1,1,0,0,0,...,1,1,1,1,1,1,1,1,1,1
20170306,6,6,6,6,4,4,4,0,0,0,...,6,6,6,6,6,6,6,6,6,6
20170405,93,93,93,93,69,69,43,0,0,0,...,80,93,93,93,93,93,93,93,93,93


In [109]:
DF['Actor2Type2Code'].value_counts()

test = DF.groupby(['SQLDATE', 'Actor2Type2Code'])['GLOBALEVENTID'].count().reset_index(name='count')
test.new_col = DF.groupby(['SQLDATE', 'Actor2Type2Code'])['GLOBALEVENTID'].transform('count')
test.head()

,SQLDATE,Actor2Type2Code,count
0,20170404,GOV,8
1,20170404,HLH,3
2,20170404,LEG,1
3,20170405,BUS,4
4,20170405,CVL,7


In [1]:
import gdelt

# Version 1 queries
gd1 = gdelt.gdelt(version=1)

results = gd2.Search(['2016 11 01'],table='events',coverage=True)
print(len(results))

ImportError: No module named gdelt

In [1]:
import pip
installed_packages = pip.get_installed_distributions()
installed_packages_list = sorted(["%s==%s" % (i.key, i.version)
     for i in installed_packages])
print(installed_packages_list)

['alabaster==0.7.9', 'anaconda-client==1.6.0', 'anaconda-navigator==1.5', 'anaconda-project==0.4.1', 'appnope==0.1.0', 'appscript==1.0.1', 'argcomplete==1.0.0', 'astroid==1.4.9', 'astropy==1.3', 'babel==2.3.4', 'backports-abc==0.5', 'backports.shutil-get-terminal-size==1.0.0', 'backports.ssl-match-hostname==3.4.0.2', 'beautifulsoup4==4.5.3', 'bitarray==0.8.1', 'blaze==0.10.1', 'bokeh==0.12.4', 'boto==2.45.0', 'bottleneck==1.2.0', 'cdecimal==2.3', 'cffi==1.9.1', 'chardet==2.3.0', 'chest==0.2.3', 'click==6.7', 'cloudpickle==0.2.2', 'clyent==1.2.2', 'colorama==0.3.7', 'conda==4.3.14', 'configobj==5.0.6', 'configparser==3.5.0', 'contextlib2==0.5.4', 'cryptography==1.7.1', 'cycler==0.10.0', 'cython==0.25.2', 'cytoolz==0.8.2', 'dask==0.13.0', 'datashape==0.5.4', 'decorator==4.0.11', 'dill==0.2.5', 'docutils==0.13.1', 'enum34==1.1.6', 'et-xmlfile==1.0.1', 'fastcache==1.0.2', 'flask-cors==3.0.2', 'flask==0.12', 'funcsigs==1.0.2', 'functools32==3.2.3.post2', 'futures==3.0.5', 'gevent==1.2.1', '